# Lesson 1 Homework: Cats vs. Dogs
The goal of this assignment is to get to the top 50% of the Dogs vs. Cats competition.
From https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/leaderboard,
this is roughly a log loss of 0.12.

My first idea is to use pretrained VGG16 from keras and train some number of fully connected layers on top.

In [9]:
# following https://keras.io/applications/#vgg16
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Dense
from keras.models import Model

import numpy as np

In [30]:
# pooling converts last layer into 2D tensor
base_model = VGG16(weights='imagenet', include_top=False, pooling="avg")

In [31]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

Add layers

In [32]:
x = base_model.output
# Fully connected layer
x = Dense(1024, activation='relu')(x)
# final outputs
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

Training setup

In [33]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all VGG layers
for layer in base_model.layers:
    layer.trainable = False

In [34]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [35]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

### Fit

In [36]:
batch_size=64

In [37]:
# Need to preprocess input here?
def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, 
                batch_size=batch_size, class_mode='categorical'):
    return gen.flow_from_directory(dirname, target_size=(224,224), 
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [38]:
#path = "data/dogscats/"
# For testing code, not enough data for anything serious
path = "data/dogscats/sample/"

In [39]:
train_batches = get_batches(path + 'train', batch_size=batch_size)
val_batches = get_batches(path + 'valid', batch_size=batch_size)

Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [40]:
imgs, labels = next(train_batches)

In [41]:
imgs.shape

(16, 224, 224, 3)

In [42]:
labels.shape

(16, 2)

In [43]:
model.fit_generator(train_batches, steps_per_epoch=10, epochs=1, 
                    validation_data=val_batches, validation_steps=10)

Epoch 1/1
10/10 [==============================] - 6s 567ms/step - loss: 7.8837 - val_loss: 8.0590


# TODO
1. Check if preprocessing needed
1. use full data
1. Get steps per epoch/batch/sample right
1. Compute accuracy?